In [1]:
import sys
sys.path.append("../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from models import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.5,), std=(0.5,))])

mnist_dset_train = torchvision.datasets.MNIST('./data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('./data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [4]:
activation_type = "tanh"
architecture = [784, 500, 500, 10]

x,y = next(iter(train_loader))
x = x.view(x.size(0),-1).to(device).T
y_one_hot = F.one_hot(y, 10).to(device).T
lambda_ = 0.9999
epsilon = 0.1#0.5
neural_lr_start = 0.1/10
neural_lr_stop = 0.0001
neural_lr_rule = "divide_by_slow_loop_index"
neural_lr_decay_multiplier = 0.1
neural_dynamic_iterations = 50

lr_start = {'ff' : 0.01, "fb": 0.001}
# lr_start = {'ff' : 0.001, "fb": 0.001}

model = CorInfoMaxV2(architecture, lambda_, epsilon, activation_type)

In [ ]:
trn_acc_list = []
tst_acc_list = []

n_epochs = 10
lr = lr_start
for epoch_ in range(n_epochs):
    lr = {'ff' : lr_start['ff'] * (0.9)**epoch_, 'fb' : lr_start['fb'] * (0.9)**epoch_}
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device).view(x.size(0),-1).T, y.to(device)
        x_ = torch.clone(x)
        #x = activation_inverse(x.view(x.size(0),-1).T, activation_type)
        #x = x.view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T
        #y_one_hot = 0.94 * y_one_hot + 0.03 * torch.ones(*y_one_hot.shape, device = device)
        _ = model.batch_step( x, y_one_hot, lr, neural_lr_start, neural_lr_stop, neural_lr_rule,
                              neural_lr_decay_multiplier, neural_dynamic_iterations, optimizer = "sgd"
                            )
#         break
#     break
    trn_acc = evaluatePC(model, train_loader, device, False, activation_type = activation_type, printing = False)
    tst_acc = evaluatePC(model, test_loader, device, False, activation_type = activation_type, printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

3000it [04:28, 11.17it/s]
2it [00:00, 14.90it/s]

Epoch : 1, Train Accuracy : 0.7247833333333333, Test Accuracy : 0.7326


3000it [04:13, 11.83it/s]
2it [00:00, 11.34it/s]

Epoch : 2, Train Accuracy : 0.7994666666666667, Test Accuracy : 0.8054


3000it [04:16, 11.69it/s]
2it [00:00, 11.06it/s]

...

3000it [04:26, 11.25it/s]
1it [00:00,  9.99it/s]

Epoch : 6, Train Accuracy : 0.83365, Test Accuracy : 0.8387


3000it [04:23, 11.39it/s]
0it [00:00, ?it/s]

Epoch : 7, Train Accuracy : 0.8355, Test Accuracy : 0.8425


2613it [03:53, 11.24it/s]

In [ ]:
torch.norm(activation_func(x, "tanh")[0] - x_.view(x_.size(0),-1).T)

In [ ]:
model = CorInfoMaxV2(architecture, lambda_, epsilon, activation_type)
lambda_ = 0.9999
epsilon = 0.1#0.5
neural_lr_start = 0.1/25
neural_lr_stop = 0.0001
neural_lr_rule = "divide_by_slow_loop_index"
neural_lr_decay_multiplier = 0.1
neural_dynamic_iterations = 50

x,y = next(iter(train_loader))
x, y = x.to(device), y.to(device)
y_one_hot = F.one_hot(y, 10).to(device).T
x = activation_inverse(x.view(x.size(0),-1).T, activation_type)

neurons = model.init_neurons(x.size(1), device = model.device)
neurons = model.fast_forward(x)

# if mode == "train":
neurons[-1] = y_one_hot.to(torch.float)

neurons = model.run_neural_dynamics(x, y_one_hot, neurons, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                                   neural_lr_decay_multiplier, neural_dynamic_iterations, 1)

In [ ]:
activation_inverse(torch.tensor([-1]), "tanh")

In [ ]:
neuron_grads = model.calculate_neural_dynamics_grad(x,y_one_hot, neurons, 1, "train")

In [ ]:
neuron_grads

In [ ]:
neurons[-2]

In [ ]:
plot_convergence_plot(trn_acc_list, xlabel = 'Number of Epochs', ylabel = 'Accuracy %',
                      title = 'PC Train Accuracy w.r.t. Epochs', 
                      figsize = (12,8), fontsize = 25, linewidth = 3)

In [ ]:
plot_convergence_plot(tst_acc_list, xlabel = 'Number of Epochs', ylabel = 'Accuracy %',
                      title = 'PC Test Accuracy w.r.t. Epochs', 
                      figsize = (12,8), fontsize = 25, linewidth = 3)

In [ ]:
plt.figure(figsize = (7,7))
plt.imshow(torch2numpy(model.B[0]["weight"]))

In [ ]:
plt.figure(figsize = (7,7))
plt.imshow(torch2numpy(model.Bsigma[1]["weight"]))

In [ ]:
model.Bsigma[1]["weight"]